# Implemeting Feature Sign Search Algorithm for Sparse Coding

In [1]:
import numpy as np

# B = Bases matrix (k x n)
#     k = dimension of bases vectors
#     n = number of vectors

# Y = Vector to encoded (k x 1) 
# X = Array of Coefficients (n x 1) 
# g = Gamma (scalar constant)
# Aim is to find X such that Y = BX (X such that BX as close to Y as possible)


def feature_sign_search(B,Y,g):
    
    #Step 1
    print 'Step1'
    (k,n) = np.shape(B)
    k1 = np.shape(Y)[0]
    assert(k == k1)
    X = np.zeros(n, dtype = np.float32)
    T = np.zeros(n, dtype = np.float32)
    #X = np.array([1,-1,0,1,2], dtype=np.float32)
    ZERO = 1e-6
    active_set = list()
    condA = True
    condB = True
    
    while condB:
        # Step 2
        print 'Step2'
        condA = True
        Z = Y - np.matmul(B,X)
        #print Z
        PD = -2*(np.matmul(Z,B))
        PDabs = np.abs(PD)
        #print PDabs
        i = np.argmax(PDabs)
        while X[i] != 0:
            PDabs[i] = 0
            i = np.argmax(PDabs)
 
        if PD[i] > g:
            T[i] = -1
            active_set.append(i)
        if PD[i] < -g:
            T[i] = 1
            active_set.append(i)
        #print T
        #print active_set
    
    
        while condA:
            # Step 3
            print 'Step3'
            p = len(active_set) # p is number of vectors we are using now
            Ba = B[:,active_set] # Ba = k x p
            Xa = X[active_set] # size(Xa) = p
            Ta = T[active_set] # size(Ta) = p
            W = np.reshape(np.matmul(np.transpose(Ba),Ba),(p,p))
            Winv = np.linalg.inv(W)
            U = np.matmul(np.transpose(Ba),Y) - (g/2)*Ta
            Xb = np.matmul(Winv,U)
            Xb = np.reshape(Xb,p)
            # ObjF = (np.linalg.norm(Y - np.matmul(Ba,Xa)))**2 + g*np.dot(Ta,Xa)
            # Above is objective function that needs to be minimized
            N = 100 # Number of points for line search
            ObjFopt = (np.linalg.norm(Y - np.matmul(Ba,Xb)))**2 + g*np.dot(Ta,Xb) #Assuming that this is optimal value of ObjF
            Xopt = Xb #Point at which objective function is optimal based on assumption
            Xprev = Xa #Starting the iteration
            for jj in range(N):
                Xcurr = Xa + (float(jj)/N)*(Xb - Xa)
                signChange = sum(np.abs(np.sign(Xprev) - np.sign(Xcurr)))
                if signChange >= 1:
                    ObjFcurr = (np.linalg.norm(Y - np.matmul(Ba,Xcurr)))**2 + g*np.dot(Ta,Xcurr)
                    if ObjFcurr < ObjFopt:
                        ObjFopt = ObjFcurr
                        Xopt = Xcurr
                Xprev = Xcurr      
            Xa = Xopt
            X[active_set] = Xa
            T = np.sign(X)
            new_active_set = list()
            for e in active_set:
                if X[e] != 0:
                    new_active_set.append(e) 
            active_set = new_active_set
            
            condA = False
            # Step 4
            print 'Step4'
            for ii in active_set:
                hii = -2*np.dot((Y - np.matmul(B,X)),B[:,ii]) + g*np.sign(X[ii])
                if abs(hii) > ZERO:
                    print abs(hii)
                    condA = True # goto step 3
            
        condB = False
        for ii in range(n):
            if ii not in active_set:
                hii = -2*np.dot((Y - np.matmul(B,X)),B[:,ii])
                if abs(hii) > g:
                    condB = True # goto step 2
    return X

$$Following \ is \ a \ small \ example \ to \ demonstrate \ the \ working \ of \ code \ and \ check \ its \ correctness $$
$$Consider \ \vec{y} \ as \ a \ 3 \ dimensional \ vector $$
$$\vec{y}=\begin{bmatrix}1 & 2 & 3 \end{bmatrix}^T$$
$$Consider \ 5 \ vectors \ in \ 3 \ dimensional \ space $$
$$\vec{b}_1=\begin{bmatrix}1 & -2 & -1 \end{bmatrix}^T$$
$$\vec{b}_2=\begin{bmatrix}2 & 1& 1 \end{bmatrix}^T$$
$$\vec{b}_3=\begin{bmatrix}1 & 1 & 1 \end{bmatrix}^T $$
$$\vec{b}_4=\begin{bmatrix}-1 & -1 & 2 \end{bmatrix}^T$$
$$\vec{b}_5=\begin{bmatrix}-1 & 0 & 1 \end{bmatrix}^T$$
$$\\$$
$$We\ want \ to \ find \ coefficients \ x_i \ s \ such \ that$$
$\\$
$$\vec{y} = x_1\vec{b}_1 + x_2\vec{b}_2 + x_3\vec{b}_3 + x_4\vec{b}_4 + x_5\vec{b}_5$$

In [2]:
B = np.array([[1,2,1,-1,-1],[-2,1,1,-1,0],[-1,1,1,2,1]])
Y = np.array([1,2,3])
g = 0.000001
X = feature_sign_search(B,Y,g)
print X

Step1
Step2
Step3
Step4
Step2
Step3
Step4
Step2
Step3
Step4
[-0.71428567  1.14285707  0.          0.57142848  0.        ]


In [3]:
print np.matmul(B,X)

[ 1.          1.99999994  2.9999997 ]


$$We \ clealy \ see \ that \ resulting \ x_i s \ give \ \vec{y} \ components \ to \ accuracy \ of \ order \ of  \ 10^{-7}$$